<a href="https://colab.research.google.com/github/yfqian95/JWAS.jl/blob/master/Aug2morning_PedigreeJWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia CSV DataFrames Random Statistics Distributions LinearAlgebra SparseArrays Printf JWAS"  # CSV DataFrames Random Statistics JWAS Distributions LinearAlgebra
JULIA_PACKAGES_IF_GPU=""
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-08-01 04:12:08 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package CSV...
Installing Julia package DataFrames...
Installing Julia package Random...
Installing Julia package Statistics...
Installing Julia package JWAS...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.7

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


chapter 5 of the e-book

In [3]:
##Aug 2 afternoon
using DataFrames, SparseArrays, LinearAlgebra
animal= ["animal1","animal2","animal3","animal4","animal5","animal6"]
sex   = ["m","f","f","m","f","f"]
breed = ["Angus","Angus","Hereford","Hereford","Angus","Angus"]
age   = [40,36,38,42,40,36]
df    = DataFrame(animal=animal,sex=sex,breed=breed,age=age,y=round.(randn(6),digits=3))

,animal,sex,breed,age,y
,String,String,String,Int64,Float64
1,animal1,m,Angus,40,-2.033
2,animal2,f,Angus,36,-1.558
3,animal3,f,Hereford,38,-1.001
4,animal4,m,Hereford,42,-0.367
5,animal5,f,Angus,40,-0.096
6,animal6,f,Angus,36,0.122


In [4]:
A=df[:,:sex]

6-element Vector{String}:
 "m"
 "f"
 "f"
 "m"
 "f"
 "f"

In [5]:
res = unique(A)

2-element Vector{String}:
 "m"
 "f"

In [6]:
dictA = Dict()                 # declare empty dictionary
for (i,s) in enumerate(res)    # fill the dictionary with the values in res, this step creates the dictionary
    dictA[s] = i
end
dictA

Dict{Any, Any} with 2 entries:
  "f" => 2
  "m" => 1

In [7]:
ii = 1:size(A,1)            # get row numbers just 1 to 6
jj = [dictA[i] for i in A]  # get column numbers using list comprehension extract M or F 
[ii jj]

6×2 Matrix{Int64}:
 1  1
 2  2
 3  2
 4  1
 5  2
 6  2

In [8]:
XA   = sparse(ii,jj,1.0) #1.0 if there is same number keep puttin in 1.0

6×2 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅ 
  ⋅   1.0
  ⋅   1.0
 1.0   ⋅ 
  ⋅   1.0
  ⋅   1.0

In [9]:
n = size(A,1)
Matrix([ones(n,1) XA])

6×3 Matrix{Float64}:
 1.0  1.0  0.0
 1.0  0.0  1.0
 1.0  0.0  1.0
 1.0  1.0  0.0
 1.0  0.0  1.0
 1.0  0.0  1.0

In [10]:
# This function returns a dictionary with the unique values in the vector "a" as the keys and their 
# sequential numbers as the associated values
# It also returns vector with the keys in sequential order. 
function mkDict(a)
  aUnique = unique(a)
  d = Dict()
  names = Array{String}(undef,size(aUnique,1))
  for (i,s) in enumerate(aUnique)
    names[i] = s
    d[s] = i
  end
  return d,names
end

mkDict (generic function with 1 method)

In [11]:
dictA,namesA   = mkDict(A)

(Dict{Any, Any}("f" => 2, "m" => 1), ["m", "f"])

In [12]:
ii = 1:size(A,1)
jj = [dictA[i] for i in A]  #list comprehension 
[ii jj]
XA = sparse(ii,jj,1.0)

6×2 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅ 
  ⋅   1.0
  ⋅   1.0
 1.0   ⋅ 
  ⋅   1.0
  ⋅   1.0

In [13]:
B = df[:,:breed]
dictB,namesB   = mkDict(B)
jj   = [dictB[i] for i in B]  #list comprehension 
ii   = 1:size(B,1)
XB   = sparse(ii,jj,1.0)

6×2 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅ 
 1.0   ⋅ 
  ⋅   1.0
  ⋅   1.0
 1.0   ⋅ 
 1.0   ⋅ 

In [14]:
CVal = df[:,:age]
CStr = fill("age",size(CVal,1))  # only one column in design matrix for age

6-element Vector{String}:
 "age"
 "age"
 "age"
 "age"
 "age"
 "age"

In [15]:
dictC,namesC   = mkDict(CStr)
jj   = [dictC[i] for i in CStr]  #list comprehension 
ii   = 1:size(CStr,1)
XC   = sparse(ii,jj,CVal) #continuous variable uses value CVal

6×1 SparseMatrixCSC{Int64, Int64} with 6 stored entries:
 40
 36
 38
 42
 40
 36

In [16]:
n = size(A,1)
Matrix([ones(n,1) XA XB XC])

6×6 Matrix{Float64}:
 1.0  1.0  0.0  1.0  0.0  40.0
 1.0  0.0  1.0  1.0  0.0  36.0
 1.0  0.0  1.0  0.0  1.0  38.0
 1.0  1.0  0.0  0.0  1.0  42.0
 1.0  0.0  1.0  1.0  0.0  40.0
 1.0  0.0  1.0  1.0  0.0  36.0

In [17]:
["intercept"; namesA; namesB; namesC]

6-element Vector{String}:
 "intercept"
 "m"
 "f"
 "Angus"
 "Hereford"
 "age"

In [18]:
firstName = "Short"
lastName = " Course"
firstName * lastName

"Short Course"

In [19]:
AB = A .*" x ".*B

6-element Vector{String}:
 "m x Angus"
 "f x Angus"
 "f x Hereford"
 "m x Hereford"
 "f x Angus"
 "f x Angus"

In [20]:
dictAB,namesAB   = mkDict(AB)
ii   = 1:size(A,1)
jj   = [dictAB[i] for i in AB]  #list comprehension 
XAB = sparse(ii,jj,1.0)

6×4 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅   1.0
  ⋅   1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅ 

In [21]:
namesAB

4-element Vector{String}:
 "m x Angus"
 "f x Angus"
 "f x Hereford"
 "m x Hereford"

In [22]:
n = size(A,1)
Matrix([ones(n,1) XA XB XAB])

6×9 Matrix{Float64}:
 1.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
 1.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0

In [23]:
["intercept"; namesA; namesB; namesAB]

9-element Vector{String}:
 "intercept"
 "m"
 "f"
 "Angus"
 "Hereford"
 "m x Angus"
 "f x Angus"
 "f x Hereford"
 "m x Hereford"

In [24]:
BVal = df[:,:age]
BStr = fill("age",size(BVal,1))

6-element Vector{String}:
 "age"
 "age"
 "age"
 "age"
 "age"
 "age"

In [25]:
AB = A.*" x ".*BStr 

6-element Vector{String}:
 "m x age"
 "f x age"
 "f x age"
 "m x age"
 "f x age"
 "f x age"

In [26]:
dAB,namesAB   = mkDict(AB)
ii    = 1:size(AB,1)
jj   = [dAB[i] for i in AB]  #list comprehension 
XAB   = sparse(ii,jj,BVal)
Matrix(XAB)

6×2 Matrix{Int64}:
 40   0
  0  36
  0  38
 42   0
  0  40
  0  36

In [27]:
Matrix([ones(n,1) XA XAB])

6×5 Matrix{Float64}:
 1.0  1.0  0.0  40.0   0.0
 1.0  0.0  1.0   0.0  36.0
 1.0  0.0  1.0   0.0  38.0
 1.0  1.0  0.0  42.0   0.0
 1.0  0.0  1.0   0.0  40.0
 1.0  0.0  1.0   0.0  36.0

In [28]:
["intercept"; namesA; namesAB]

5-element Vector{String}:
 "intercept"
 "m"
 "f"
 "m x age"
 "f x age"

In [29]:
# Test code for qualitative factor
factor = "sex"
cov    = false
data = df[:,Symbol(factor)]

n = size(data,1)
if cov==false
    str = data
    val = 1.0
else 
    str = fill(factor,n)
    val = data
end

dict,colNames   = mkDict(str)
ii    = 1:n                    # row numbers 
jj   = [dict[i] for i in str]  # column numbers
X    = sparse(ii,jj,val)
Matrix(X)    

6×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0

In [30]:
# Test code for quantitative factor
factor = "age"
cov    = true

data = df[:,Symbol(factor)]
n = size(data,1)
if cov==false
    str = data
    val = 1.0
else 
    str = fill(factor,n)
    val = data
end

dict,colNames   = mkDict(str)
ii    = 1:n                    # row numbers 
jj   = [dict[i] for i in str]  # column numbers
X    = sparse(ii,jj,val)
Matrix(X)

6×1 Matrix{Int64}:
 40
 36
 38
 42
 40
 36

In [31]:
function getX(factor,df;cov=false)
    data = df[:,Symbol(factor)]
    n = size(data,1)
    if cov==false
        str = data
        val = 1.0
    else 
        str = fill(factor,n)
        val = data
    end

    dict,names   = mkDict(str)
    ii    = 1:n                    # row numbers 
    jj   = [dict[i] for i in str]  # column numbers
    X    = sparse(ii,jj,val)    
end        

getX (generic function with 1 method)

In [32]:
function getX(factors,covs,df)
    n = size(df,1)
    if covs[1] == false
        str = df[:,Symbol(factors[1])]
        val = 1.0
    else
        str = fill(factors[1],n) 
        val = df[:,Symbol(factors[1])]    
    end       

    for i in 2:length(factors)
        if covs[i] == false
            str = str .*" x ".*df[:,Symbol(factors[i])]
            val = val .* 1.0 
        else
            str = str .*" x ".*fill(factors[i],n) 
            val = val .* df[:,Symbol(factors[i])]    
        end 
    end 
    dict,colNames   = mkDict(str)
    ii    = 1:n                    # row numbers 
    jj   = [dict[i] for i in str]  # column numbers
    X    = sparse(ii,jj,val)
    return X,colNames        
end            

getX (generic function with 2 methods)

In [33]:
factors = ["sex", "breed"]
covs =[false, false]
X,colNames = getX(factors,covs,df)
Matrix(X)

6×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0

In [ ]:
factors = ["age", "age"] #not a linear regression and this is not age interact with age; this is a quadratic curve, fitting a curve of age
covs =[true, true]
X,colNames = getX(factors,covs,df)
Matrix(X)

In [36]:
modelTerm   = "sex * breed"
covariables = ["age"];

In [37]:
factors = strip.(split(modelTerm,"*"))

2-element Vector{SubString{String}}:
 "sex"
 "breed"

In [38]:
covs = [i in covariables for i in factors]

2-element Vector{Bool}:
 0
 0

In [39]:
function getX(modelTerm,covariables,df)
    n = size(df,1)
    if modelTerm == "intercept"
        X = ones(n,1)
        colNames = ["intercept"]
        return X,colNames
    end
    factors = strip.(split(modelTerm,"*"))
    covs = [i in covariables for i in factors]
    
    if covs[1] == false
        str = string.(df[:,Symbol(factors[1])])
        val = 1.0
    else
        str = fill(factors[1],n) 
        val = df[:,Symbol(factors[1])]    
    end       

    for i in 2:length(factors)
        if covs[i] == false
            str = str .*" x ".*string.(df[:,Symbol(factors[i])])
            val = val .* 1.0 
        else
            str = str .*" x ".*fill(factors[i],n) 
            val = val .* df[:,Symbol(factors[i])]    
        end 
    end 
    dict,colNames   = mkDict(str)
    ii = 1:n                     # row numbers 
    jj = [dict[i] for i in str]  # column numbers
    X  = sparse(ii,jj,val)
    return X, strip(modelTerm)*": ".*colNames   
end

getX (generic function with 2 methods)

In [40]:
modelTerm   = "sex * breed"
covariables = ["age"]
X,colNames = getX(modelTerm,covariables,df)
Matrix(X)

6×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0

In [41]:
colNames

4-element Vector{String}:
 "sex * breed: m x Angus"
 "sex * breed: f x Angus"
 "sex * breed: f x Hereford"
 "sex * breed: m x Hereford"

In [42]:
modelTerm = "breed*age"
X,colNames = getX(modelTerm,covariables,df)
Matrix(X)

6×2 Matrix{Float64}:
 40.0   0.0
 36.0   0.0
  0.0  38.0
  0.0  42.0
 40.0   0.0
 36.0   0.0

In [43]:
colNames

2-element Vector{String}:
 "breed*age: Angus x age"
 "breed*age: Hereford x age"

In [44]:
modelTerm = "sex"
X,colNames = getX(modelTerm,covariables,df)
Matrix(X)

6×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0

In [45]:
modelTerm = "intercept"
X,colNames = getX(modelTerm,covariables,df)
Matrix(X)

6×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [46]:
modelEq = "y = intercept + sex + breed + sex*breed + age"

"y = intercept + sex + breed + sex*breed + age"

In [47]:
modelParts = strip.(split(modelEq,"="))

2-element Vector{SubString{String}}:
 "y"
 "intercept + sex + breed + sex*breed + age"

In [48]:
depVar = modelParts[1]
modelTerms = strip.(split(modelParts[2],"+"))

5-element Vector{SubString{String}}:
 "intercept"
 "sex"
 "breed"
 "sex*breed"
 "age"

In [49]:
X,colNames = getX(modelTerms[1],covariables,df)
Matrix(X)

6×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [50]:
function getLhsRhs(modelEq,covariables,df)
    modelParts = strip.(split(modelEq,"="))
    depVar = modelParts[1]
    y = df[:,Symbol(depVar)]
    modelTerms = strip.(split(modelParts[2],"+"))
    X,colNames = getX(modelTerms[1],covariables,df)
    for i = 2:size(modelTerms,1)
        Xi,namesi = getX(modelTerms[i],covariables,df)
        X = [X Xi]
        colNames = [colNames; namesi]
    end
    return X'X,X'y,colNames
end

getLhsRhs (generic function with 1 method)

In [51]:
lhs,rhs,colNames = getLhsRhs(modelEq,covariables,df) #df=dataframe 
#X'X=lhs (left hand side, the incident matrix, every x); rhs=X'Y
[colNames Matrix(lhs) rhs]

10×12 Matrix{Any}:
 "intercept"                  6.0  …    3.0   1.0   1.0   232.0    -4.933
 "sex: m"                     2.0       0.0   0.0   1.0    82.0    -2.4
 "sex: f"                     4.0       3.0   1.0   0.0   150.0    -2.533
 "breed: Angus"               4.0       3.0   0.0   0.0   152.0    -3.565
 "breed: Hereford"            2.0       0.0   1.0   1.0    80.0    -1.368
 "sex*breed: m x Angus"       1.0  …    0.0   0.0   0.0    40.0    -2.033
 "sex*breed: f x Angus"       3.0       3.0   0.0   0.0   112.0    -1.532
 "sex*breed: f x Hereford"    1.0       0.0   1.0   0.0    38.0    -1.001
 "sex*breed: m x Hereford"    1.0       0.0   0.0   1.0    42.0    -0.367
 "age: age"                 232.0     112.0  38.0  42.0  9000.0  -190.308

In [52]:
QRLhs = qr(lhs) 
sol = QRLhs\rhs

10-element Vector{Float64}:
 -6.909999999996091
  0.012000000000411083
  0.0
  0.5939999999999267
  0.0
 -1.9490000000001324
  0.0
  0.0
  0.0
  0.15549999999989733